In [ ]:
from splunk_tools import SplunkTools
import asyncio
savedsearches = ["Windows Event For Service Disabled",
                 "Detect New Local Admin account",
                 "ESCU Network Share Discovery Via Dir Command Rule",
                 "Known Services Killed by Ransomware",
                 "Non Chrome Process Accessing Chrome Default Dir",
                 "Kerberoasting spn request with RC4 encryption",
                 "Clop Ransomware Known Service Name",
                 "WinEvent Scheduled Task Created Within Public Path"]

splunk = SplunkTools(savedsearches, 1)

logtypes_list = [('wineventlog:security', '4698')]
# logtypes_list = [('wineventlog:system', '1'), ('wineventlog:system', '16'), ('wineventlog:system', '1085'), ('wineventlog:security', '5061'), ('wineventlog:system', '7000'), ('wineventlog:system', '566'), ('wineventlog:system', '107'), ('wineventlog:system', '55'), ('wineventlog:security', '4663'), ('wineventlog:system', '7038'), ('wineventlog:system', '24'), ('wineventlog:security', '4625'), ('wineventlog:system', '7031'), ('wineventlog:security', '4662'), ('wineventlog:system', '7040'), ('wineventlog:security', '4672'), ('wineventlog:system', '1112'), ('wineventlog:system', '8015'), ('wineventlog:system', '101'), ('wineventlog:security', '4702'), ('wineventlog:system', '44'), ('wineventlog:security', '4735'), ('wineventlog:system', '37'), ('wineventlog:security', '4697'), ('wineventlog:security', '5059'), ('wineventlog:system', '1500'), ('wineventlog:system', '7036'), ('wineventlog:system', '1501'), ('wineventlog:system', '131'), ('wineventlog:system', '187'), ('wineventlog:security', '4732'), ('wineventlog:system', '103'), ('wineventlog:system', '32'), ('wineventlog:system', '35'), ('wineventlog:security', '4799'), ('wineventlog:security', '5058'), ('wineventlog:security', '4648'), ('wineventlog:security', '5379'), ('wineventlog:security', '4634'), ('wineventlog:system', '7'), ('wineventlog:system', '129'), ('wineventlog:security', '5038'), ('wineventlog:security', '4907'), ('wineventlog:security', '4624'), ('wineventlog:system', '27'), ('wineventlog:system', '10016'), ('wineventlog:system', '7045'), ('wineventlog:system', '6'), ('wineventlog:system', '108'), ('wineventlog:security', '4698'), ('wineventlog:security', '5140'), ('wineventlog:system', '12'), ('wineventlog:security', '4627'), ('wineventlog:system', '6013'), ('wineventlog:security', '4769')]
# find one sample of each logtype in splunk in all time and save as raw
async def find(logtype, eventcode):
    query = f"index=main source={logtype} EventCode={eventcode} | head 1"
    return await splunk.execute_query(f"find_one_{logtype}_{eventcode}", query, 0, "now")
    
results = await asyncio.gather(*[find(logtype, eventcode) for logtype, eventcode in logtypes_list])

/home/shouei/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
log_samples = {}
for metric, result in results:
    result = result[0]
    source = result['source'] 
    eventcode = result['EventCode']
    raw = result['_raw']
    log_samples[(source, eventcode)] = raw

log_samples   

{('WinEventLog:Security',
  '4698'): '06/23/2025 11:09:03 AM\nLogName=Security\nEventCode=4698\nEventType=0\nComputerName=LB-003-25.auth.ad.bgu.ac.il\nSourceName=Microsoft Windows security auditing.\nType=Information\nRecordNumber=3253237\nKeywords=Audit Success\nTaskCategory=Other Object Access Events\nOpCode=Info\nMessage=A scheduled task was created.\r\n\r\nSubject:\r\n\tSecurity ID:\t\tNT AUTHORITY\\SYSTEM\r\n\tAccount Name:\t\tLB-003-25$\r\n\tAccount Domain:\t\tBGU-USERS\r\n\tLogon ID:\t\t0x3E7\r\n\r\nTask Information:\r\n\tTask Name: \t\t\\copy user data\r\n\tTask Content: \t\t<?xml version="1.0" encoding="UTF-16"?>\r\n<Task version="1.2" xmlns="http://schemas.microsoft.com/windows/2004/02/mit/task">\r\n  <RegistrationInfo>\r\n    <Author>BGU-USERS\\ilyadr</Author>\r\n    <URI>\\copy user data</URI>\r\n  </RegistrationInfo>\r\n  <Triggers>\r\n    <TimeTrigger>\r\n      <StartBoundary>2025-06-23T11:09:08</StartBoundary>\r\n      <EndBoundary>2025-06-23T11:10:03</EndBoundary>\r\n  

In [1]:
from datetime import datetime
import re
from env_utils import clean_env
# validate raw logs by send them to splunk and search for them
async def validate(source, eventcode, raw):
    for i, log in enumerate(raw):
        # change date using regex
        log = re.sub(r'^\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2} [APM]{2}', datetime.now().strftime("%m/%d/%Y %I:%M:%S %p"), log)
        raw[i] = log
    # send raw to splunk
    print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Writing {len(raw)} logs to splunk for validation: {source} {eventcode}")
    splunk.write_logs_to_monitor(raw, source.lower())
    print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - finished writing logs to splunk for validation: {source} {eventcode}")
    results = []
    while len(results) == 0 or int(results[1][0]['count']) < len(raw):
        # wait for splunk to index the logs
        await asyncio.sleep(1)       
        # search for the raw in splunk
        query = f"index=main source={source} EventCode={eventcode} earliest=-10m host=dt-splunk|stats count"
        print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Starting validation query for {source} {eventcode}")
        results = await splunk.execute_query(f"validate_{source}_{eventcode}", query, 0, "now")
        print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Finished validation query for {source} {eventcode}")
        print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Validating {source} {eventcode} - found {int(results[1][0]['count'])} results")
    return results

async def validate_deletion(source, eventcode):
    # delete the logs from splunk
    print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Deleting logs from splunk for validation: {source} {eventcode}")
    clean_env(splunk, time_range="2m")
    print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Finished deleting logs from splunk for validation: {source} {eventcode}")
    results = []
    while len(results) == 0 or int(results[1][0]['count']) > 0:
        # wait for splunk to delete the logs
        await asyncio.sleep(1)
        # search for the raw in splunk
        query = f"index=main source={source} EventCode={eventcode} earliest=-10m host=dt-splunk|stats count"
        print(f"Time {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')} - Starting validation deletion query for {source} {eventcode}")
        results = await splunk.execute_query(f"validate_deletion_{source}_{eventcode}", query, 0, "now")
        print(results)
        print(f"Time {datetime.now().strftime('%m/%d/%Y %I:M:%S %p')} - Finished validation deletion query for {source} {eventcode}")
        print(f"Time {datetime.now().strftime('%m/%d/%Y %I:M:%S %p')} - Validating deletion of {source} {eventcode} - found {int(results[1][0]['count'])} results")
    return results
    


In [ ]:
results = await asyncio.gather(*[validate(source, eventcode, [raw]) for (source, eventcode), raw in log_samples.items()])


In [4]:
results

[(QueryMetrics(search_name='validate_WinEventLog:Security_4698', results_count=1, execution_time=2.049, cpu=1.9, io_metrics={'read_count': 0, 'write_count': 0, 'read_bytes': 0, 'write_bytes': 0}, start_time=0.0, end_time=0.0),
  [{'EventCode': '4698',
    'Message': 'A scheduled task was created.\r\n\r\nSubject:\r\n\tSecurity ID:\t\tNT AUTHORITY\\SYSTEM\r\n\tAccount Name:\t\tLB-003-25$\r\n\tAccount Domain:\t\tBGU-USERS\r\n\tLogon ID:\t\t0x3E7\r\n\r\nTask Information:\r\n\tTask Name: \t\t\\copy user data\r\n\tTask Content: \t\t<?xml version="1.0" encoding="UTF-16"?>\r\n<Task version="1.2" xmlns="http://schemas.microsoft.com/windows/2004/02/mit/task">\r\n  <RegistrationInfo>\r\n    <Author>BGU-USERS\\ilyadr</Author>\r\n    <URI>\\copy user data</URI>\r\n  </RegistrationInfo>\r\n  <Triggers>\r\n    <TimeTrigger>\r\n      <StartBoundary>2025-06-23T11:09:08</StartBoundary>\r\n      <EndBoundary>2025-06-23T11:10:03</EndBoundary>\r\n      <Enabled>true</Enabled>\r\n    </TimeTrigger>\r\n  </T

In [7]:
import os
path=r"/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/logs_resource/"
for logtype, eventcode in log_samples:
    file_name = f"{logtype.lower()}_{eventcode}_notrigger.txt"
    if not os.path.exists(file_name):
        with open(path+file_name, "w") as f:
            f.write(log_samples[(logtype, eventcode)])
    

In [3]:
# check ingestion and deletion latency
from splunk_tools import SplunkTools
savedsearches = ["Windows Event For Service Disabled",
                 "Detect New Local Admin account",
                 "ESCU Network Share Discovery Via Dir Command Rule",
                 "Known Services Killed by Ransomware",
                 "Non Chrome Process Accessing Chrome Default Dir",
                 "Kerberoasting spn request with RC4 encryption",
                 "Clop Ransomware Known Service Name",
                 "WinEvent Scheduled Task Created Within Public Path"]

splunk = SplunkTools(savedsearches, 1)
import os
import time
import asyncio
resource_path=r"/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/logs_resource/"
log_types = [('wineventlog:security', '4624')]
volumes = [5000]
for log_type, event_code in log_types:
    for volume in volumes:
        # ingest logs
        log_file = f"{resource_path}{log_type.lower()}_{event_code}_notrigger.txt"
        if os.path.exists(log_file):
            with open(log_file, "r") as f:
                text = f.read()
                log = text.split('\n[EOF]\n')
        logs = [log[0] for _ in range(volume)]
        # timer
        start_time = time.time()
        results = await asyncio.gather(validate(log_type, event_code, logs))
        end_time = time.time()
        ingestion_time = end_time - start_time
        print(f"Ingestion time for {log_type} with event code {event_code} and volume {volume}: {ingestion_time:.2f} seconds")
        # start_time = time.time()
        # results = await asyncio.gather(validate_deletion(log_type, event_code))
        # end_time = time.time()
        # deletion_time = end_time - start_time
        # print(f"Deletion time for {log_type} with event code {event_code}: {deletion_time:.2f} seconds")

Time 08/08/2025 03:06:37 PM - Writing 5000 logs to splunk for validation: wineventlog:security 4624
Time 08/08/2025 03:06:37 PM - finished writing logs to splunk for validation: wineventlog:security 4624
Time 08/08/2025 03:06:38 PM - Starting validation query for wineventlog:security 4624



Time 08/08/2025 03:06:42 PM - Finished validation query for wineventlog:security 4624
Time 08/08/2025 03:06:42 PM - Validating wineventlog:security 4624 - found 5000 results
Ingestion time for wineventlog:security with event code 4624 and volume 5000: 4.72 seconds
